In [2]:
from collections import Counter, OrderedDict, defaultdict
import copy
import glob
from itertools import izip
import itertools
import os
import subprocess
import tempfile

from IPython.core.display import HTML, Image
import itertools
import pandas as pd
from matplotlib import gridspec
from matplotlib_venn import venn2, venn3
import numpy as np
import scipy
import seaborn as sns

from clipper.src import CLIP_analysis
from clipper.src import get_genomic_regions
import flotilla
from gscripts import GO
from gscripts.general import dataviz
from gscripts.general import parsers
from gscripts.general import region_helpers
from gscripts.rnaseq.helpers import counts_to_rpkm

#img_dir = "/nas3/gpratt/Dropbox/TAF15/Data/mouse_integration"


:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [8]:
count_df = pd.read_table("/home/gpratt/projects/cryptic_exons/analysis/av_resequencing_v3/all_counts.txt", skiprows=1, index_col=0)
count_df.columns = [os.path.basename(item).split(".")[0] for item in count_df.columns]
count_df[count_df.columns[5:]].to_csv("/home/gpratt/Dropbox/cryptic_splicing/data/human_data/important_counts.csv")

In [9]:
metadata = pd.read_csv("/home/gpratt/Dropbox/cryptic_splicing/data/human_data/SampleInfo.csv")


In [10]:
metadata

,sample_id,cell_type,exp,stress,outlier,phenotype,paired_with_stress,rbp,mutation,genotype
0,AV_47d_1_S50_L008_R1_001,MN,AV_47d_1,False,False,AV_47d_1,True,TDP,G298S,AV_47d
1,AV_47d_puro_1_S51_L008_R1_001,MN,AV_47d_puro_1,True,False,AV_47d_puro_1,True,TDP,G298S,AV_47d
2,AV_ALS17_5_1_S45_L007_R1_001,MN,AV_ALS17_5_1,False,False,AV_ALS17_5_1,True,WT,N352S,AV_ALS17_5
3,AV_ALS17_5_puro_1_S46_L007_R1_001,MN,AV_ALS17_5_puro_1,True,False,AV_ALS17_5_puro_1,True,WT,N352S,AV_ALS17_5
4,AV_CVB_S56_L008_R1_001,MN,AV_CVB,False,False,AV_CVB,True,WT,WT,AV_CVB
5,AV_CVB_puro_S49_L007_R1_001,MN,AV_CVB_puro,True,False,AV_CVB_puro,True,WT,WT,AV_CVB
6,AV_GY6_2_1_S43_L007_R1_001,MN,AV_GY6_2_1,False,False,AV_GY6_2_1,True,TDP,R521G,AV_GY6_2
7,AV_GY6_2_puro_1_S44_L007_R1_001,MN,AV_GY6_2_puro_1,True,False,AV_GY6_2_puro_1,True,TDP,R521G,AV_GY6_2
8,AV_Kin1ALS17_3_S47_L007_R1_001,MN,AV_Kin1ALS17_3,False,False,AV_Kin1ALS17_3,True,TDP,WT,AV_Kin1ALS17_3
9,AV_Kin1ALS17_3_puro_S48_L007_R1_001,MN,AV_Kin1ALS17_3_puro,True,False,AV_Kin1ALS17_3_puro,True,TDP,WT,AV_Kin1ALS17_3
